In [1]:
using WAV
using SignalOperators
using SignalOperators.Units # allows the use of dB, Hz, s etc... as unitful values

# a pure tone 20 dB below a power 1 signal, with on and off ramps (for
# a smooth onset/offset)
sound1 = Signal(sin,ω=1kHz) |> Until(5s) |> Ramp |> Normpower |> Amplify(-20dB)

sound1 |> ToFramerate(44.1kHz) |> sink("example.wav")

# a sound defined by a file, matching the overall power to that of sound1
sound2 = "example.wav" |> Normpower |> Amplify(-20dB)

# a 1kHz sawtooth wave
sound3 = Signal(ϕ -> ϕ-π,ω=1kHz) |> Ramp |> Normpower |> Amplify(-20dB)

# a 5 Hz amplitude modulated noise
sound4 = randn |>
    Amplify(Signal(ϕ -> 0.5sin(ϕ) + 0.5,ω=5Hz)) |>
    Until(5s) |> Normpower |> Amplify(-20dB)

# a 1kHz tone surrounded by a notch noise
SNR = 5dB
x = Signal(sin,ω=1kHz) |> Until(1s) |> Ramp |> Normpower |> Amplify(-20dB + SNR)
y = Signal(randn) |> Until(1s) |> Filt(Bandstop,0.5kHz,2kHz) |> Normpower |>
  Amplify(-20dB)
scene = Mix(x,y)

# write all of the signals to a single file, at 44.1 kHz
# Append(sound1,sound2,sound3,sound4,scene) |> ToFramerate(44.1kHz) |> sink("examples.wav")

### Cannot Append to the end of an infinite signal (Error)

Signal(sin,ω=1000) |> Until(1 s) |> Amplify(RampOnFn(10 ms)) |>
    Amplify(RampOffFn(10 ms)) |> Normpower |> Amplify(-15.0 dB) |>
    Mix(randn |> Until(1 s) |>
            Filt(Bandstop,0.5 kHz,2 kHz) |> Normpower |> Amplify(-20.0 dB))